email agent
- authenticates user
    - only then are they allowed into the "inbox"
    - dynamic tools and prompt on the condition of there being an email and password in state that match hardcoded
- checks "inbox"
    - email in tool
- sends emails
    - human in the loop

In [1]:
# Import required modules
from dotenv import load_dotenv

import operator
from typing import Annotated, TypedDict, Literal, List, Union
from dataclasses import dataclass, asdict

from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, ToolMessage, SystemMessage, BaseMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.prebuilt import InjectedState

In [2]:
# load_dotenv
load_dotenv()

True

In [3]:
# Setup Model
model = ChatOllama(model="llama3.1:8b", temperature=0)

In [4]:
# Define Context & State
@dataclass
class EmailContext:
    email_address: str = "julie@example.com"
    password: str = "password123"

class AuthenticatedState(TypedDict):
    # This ensures messages are appended, not overwritten
    messages: Annotated[List[BaseMessage], add_messages]
    authenticated: bool
    context: dict  # Using dict here prevents Pydantic/Serialization errors

In [5]:
# Define Tools
@tool
def check_inbox() -> str:
    """Check the inbox for recent emails."""
    return """
    Hi Julie, 
    I'm going to be in town next week and was wondering if we could grab a coffee?
    - best, Jane (jane@example.com)
    """

@tool
def send_email(to: str, subject: str, body: str) -> str:
    """Send a response email."""
    return f"Email sent to {to} with subject {subject} and body {body}"

@tool
def authenticate(
    email: str, 
    password: str, 
    state: Annotated[dict, InjectedState] # Correctly inject the whole state
) -> str:
    """Authenticate the user with email and password. This is required first."""
    # Safety check: Get context from state safely
    context = state.get("context", {})
    
    # Access as dictionary keys
    if email == context.get("email_address") and password == context.get("password"):
        return "SUCCESS: You are now authenticated."
    return "FAILURE: Incorrect credentials."

In [6]:
# Logic for Dynamic Tool Filtering
def call_model(state: AuthenticatedState):
    """
    Decides available tools and sets the persona based on authentication.
    Forces the model to be descriptive about email drafts before sending.
    """
    is_auth = state.get("authenticated", False)
    
    if is_auth:
        # System prompt for authenticated users
        sys_msg = SystemMessage(content=(
            "You are a helpful assistant with access to the user's email. "
            "You are currently AUTHENTICATED.\n\n"
            "GUIDELINES:\n"
            "1. To see emails, call 'check_inbox'. Do not invent emails.\n"
            "2. When asked to reply, first explain your plan to the user and show the draft text.\n"
            "3. Then, call the 'send_email' tool. The system will pause for user approval "
            "before the tool actually executes."
        ))
        tools = [check_inbox, send_email]
    else:
        # System prompt for unauthenticated users
        sys_msg = SystemMessage(content=(
            "You are currently UNAUTHENTICATED.\n\n"
            "You MUST call the 'authenticate' tool with the user's email and password "
            "before you can perform any other actions. Do not apologize or explain, "
            "just call the tool if the credentials are provided."
        ))
        tools = [authenticate]
    
    # Bind the allowed tools for this specific state
    llm_with_tools = model.bind_tools(tools)
    
    # We pass the system message first, followed by the conversation history
    response = llm_with_tools.invoke([sys_msg] + state["messages"])
    
    return {"messages": [response]}

In [7]:
# Logic to update authentication status
def auth_checker(state: AuthenticatedState):
    last_message = state["messages"][-1]
    # If the last tool output contains 'SUCCESS', update the 'authenticated' bit
    if isinstance(last_message, ToolMessage) and "SUCCESS" in last_message.content:
        return {"authenticated": True}
    return {}

In [8]:
# Build the Graph
workflow = StateGraph(AuthenticatedState)

workflow.add_node("agent", call_model)
workflow.add_node("tools", ToolNode([authenticate, check_inbox, send_email]))
workflow.add_node("auth_checker", auth_checker)

workflow.add_edge(START, "agent")
workflow.add_conditional_edges("agent", tools_condition)
workflow.add_edge("tools", "auth_checker")
workflow.add_edge("auth_checker", "agent")

app = workflow.compile(checkpointer=InMemorySaver())

In [9]:
# Execute
config = {"configurable": {"thread_id": "1"}}

# Convert dataclass to dict for safe serialization
initial_context = asdict(EmailContext())

inputs = {
    "messages": [HumanMessage(content="Please login as julie@example.com / password123 and check my inbox")],
    "context": initial_context,
    "authenticated": False
}

for event in app.stream(inputs, config=config, stream_mode="values"):
    if "messages" in event:
        message = event["messages"][-1]
        message.pretty_print()

================================ Human Message =================================

Please login as julie@example.com / password123 and check my inbox
================================== Ai Message ==================================
Tool Calls:
  authenticate (5a280c82-ffb9-4248-a620-74c6f3f638a9)
 Call ID: 5a280c82-ffb9-4248-a620-74c6f3f638a9
  Args:
    email: julie@example.com
    password: password123
================================= Tool Message =================================
Name: authenticate

SUCCESS: You are now authenticated.
================================= Tool Message =================================
Name: authenticate

SUCCESS: You are now authenticated.
================================== Ai Message ==================================

{"name": "check_inbox"}


In [10]:
# Request reply to accept
run_agent("Draft a reply to Jane saying I'm free on Wednesday at 10am")

NameError: name 'run_agent' is not defined

In [ ]:
# Request reply to regect
run_agent("Draft a reply to Jane saying I'm not free next week")

In [ ]:
# Request reply with choice
run_agent("Draft a reply to Jane saying I'm free on Tuesday and Wednesday morning, which is good")